In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi
!pip install tqdm

In [3]:
import json
with open('/Users/Lu/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [5]:
help(yelp_api.search_query)

Help on method search_query in module yelpapi.yelpapi:

search_query(**kwargs) method of yelpapi.yelpapi.YelpAPI instance
    Query the Yelp Search API.
    
    documentation: https://www.yelp.com/developers/documentation/v3/business_search
    
    required parameters:
        * one of either:
            * location - text specifying a location to search for
            * latitude and longitude



In [6]:
# set our API call parameters 
LOCATION = 'St Louis, MO'
TERM = 'BBQ'

In [7]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_STL_BBQ.json"
JSON_FILE

'Data/results_in_progress_STL_BBQ.json'

In [8]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [10]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)

[i] Data/results_in_progress_STL_BBQ.json not found. Saving empty list to new file.


In [11]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [12]:
# use our yelp_api variable's search_query method to perform our API call
search_results = yelp_api.search_query(location=LOCATION,
                                       term=TERM,
                                       offset=n_results)
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [13]:
## How many results total?
total_results = search_results['total']
total_results

1800

In [14]:
## How many did we get the details for?
results_per_page = len(search_results['businesses'])
results_per_page

20

In [15]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((search_results['total']-n_results)/ results_per_page)
n_pages

90

In [16]:
# join new results with old list with extend and save to file
previous_results.extend(search_results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [18]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    ## append new results and save to file
    previous_results.extend(search_results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/90 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [19]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,I_3LMZ_1m2mzR0oLIOePIg,pappys-smokehouse-saint-louis-3,Pappy's Smokehouse,https://s3-media3.fl.yelpcdn.com/bphoto/_U0uWo...,False,https://www.yelp.com/biz/pappys-smokehouse-sai...,4385,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 38.635146597249, 'longitude': -90...","[delivery, pickup]",$$,"{'address1': '3106 Olive St', 'address2': '', ...",+13145354340,(314) 535-4340,2599.211156
1,vWjME92WddneF98IqzaNfQ,sugarfire-smoke-house-saint-louis-2,Sugarfire Smoke House,https://s3-media4.fl.yelpcdn.com/bphoto/lFajL-...,False,https://www.yelp.com/biz/sugarfire-smoke-house...,1843,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 38.673459, 'longitude': -90.367448}",[delivery],$$,"{'address1': '9200 Olive Blvd', 'address2': ''...",+13149972301,(314) 997-2301,11032.124217
2,NHb6QTrJOnyIj-Zd0fYzFg,bogarts-smokehouse-saint-louis-4,Bogart's Smokehouse,https://s3-media2.fl.yelpcdn.com/bphoto/gNo_J2...,False,https://www.yelp.com/biz/bogarts-smokehouse-sa...,1549,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 38.61098, 'longitude': -90.20286}","[delivery, pickup]",$$,"{'address1': '1627 S 9th St', 'address2': '', ...",+13146213107,(314) 621-3107,5849.887615
3,WCdSajl5Q0qywpv7K5jHdQ,sugarfire-smoke-house-saint-louis-4,Sugarfire Smoke House,https://s3-media4.fl.yelpcdn.com/bphoto/UmoHh1...,False,https://www.yelp.com/biz/sugarfire-smoke-house...,1348,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.0,"{'latitude': 38.63077, 'longitude': -90.18975}",[delivery],$$,"{'address1': '605 Washington Ave', 'address2':...",+13147978485,(314) 797-8485,5294.089599
4,R8t9g5nvi7VFyS8zsgmj8Q,salt-smoke-saint-louis,Salt + Smoke,https://s3-media2.fl.yelpcdn.com/bphoto/tZgXfN...,False,https://www.yelp.com/biz/salt-smoke-saint-loui...,2152,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 38.6561019915198, 'longitude': -9...","[delivery, pickup]",$$,"{'address1': '6525 Delmar Blvd', 'address2': '...",+13147270200,(314) 727-0200,5395.921288


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,cTvzHrIlCBMLA_AceItNZQ,tootie-s-on-washington-ave-st-louis,Tootie’s on Washington Ave,https://s3-media1.fl.yelpcdn.com/bphoto/jHrA2w...,False,https://www.yelp.com/biz/tootie-s-on-washingto...,9,"[{'alias': 'southern', 'title': 'Southern'}, {...",4.5,"{'latitude': 38.63420964602558, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '1933 Washington Ave', 'address2'...",+13142269095,(314) 226-9095,3739.700558
996,R_u4QTal0d5f-FHr7BUXcw,smoking-barrels-bbq-st-louis-st-louis,Smoking Barrels BBQ - St. Louis,https://s3-media3.fl.yelpcdn.com/bphoto/v9jrgg...,False,https://www.yelp.com/biz/smoking-barrels-bbq-s...,365,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.0,"{'latitude': 38.5786660097518, 'longitude': -9...","[delivery, pickup]",$$,"{'address1': '5641 S Kingshighway Blvd', 'addr...",+13146699455,(314) 669-9455,8957.613668
997,nwN92Uje-xIKE5voPTTvBQ,beast-craft-bbq-belleville,BEAST Craft BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/GA_8Fb...,False,https://www.yelp.com/biz/beast-craft-bbq-belle...,685,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 38.50212, 'longitude': -89.98536}","[delivery, pickup]",$$,"{'address1': '20 S Belt W', 'address2': None, ...",+16182579000,(618) 257-9000,28012.824351
998,F_uCny898hCrK1pByKLkaw,sugarfire-o-fallon-2,Sugarfire,https://s3-media2.fl.yelpcdn.com/bphoto/meezmT...,False,https://www.yelp.com/biz/sugarfire-o-fallon-2?...,145,"[{'alias': 'smokehouse', 'title': 'Smokehouse'}]",3.5,"{'latitude': 38.58113086987757, 'longitude': -...",[delivery],$$,"{'address1': '1425 N Green Mount Rd', 'address...",+16183603473,(618) 360-3473,28392.888373
999,1o9IdLrFGMS2o5Y8uXKC0Q,adams-smokehouse-saint-louis,Adam's Smokehouse,https://s3-media3.fl.yelpcdn.com/bphoto/yeqxli...,False,https://www.yelp.com/biz/adams-smokehouse-sain...,378,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 38.6093667, 'longitude': -90.29013}","[delivery, pickup]",$$,"{'address1': '2819 Watson Rd', 'address2': '',...",+13148759890,(314) 875-9890,6357.980247


In [21]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

600

In [22]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [23]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_STL_BBQ.csv.gz', compression='gzip',index=False)